In [1]:
# Copyright © 2018-2021 Intel Corporation All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions
# are met:
#   * Redistributions of source code must retain the above copyright
#     notice, this list of conditions and the following disclaimer.
#   * Redistributions in binary form must reproduce the above copyright
#     notice, this list of conditions and the following disclaimer in
#     the documentation and/or other materials provided with the
#     distribution.
#   * Neither the name of Intel Corporation nor the names of its
#     contributors may be used to endorse or promote products derived
#     from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
# A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
# OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
# SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
# LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
# DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
# THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [2]:
%config IPCompleter.greedy=True
import sys
import os
sys.path.append('/home/pplank/nxsdk-nxsdk-submission/nxsdk-nxsdk/')
import numpy as np
import pickle
import matplotlib.pyplot as plt
import logging
from logging import Filter, StreamHandler
import nxsdk.api.n2a as nx
from nxsdk.utils.env_var_context_manager import setEnvWithinContext
from nxsdk.graph.processes.phase_enums import Phase
from nxsdk.api.enums.api_enums import ProbeParameter
from nxsdk.graph.monitor.probes import PerformanceProbeCondition
from nxsdk_modules.lsnn.apps.relnet.data.loihi_chip_optimum_network_from_data import get_relational_network_placement_on_loihi

from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import perform_LSNN_placement
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import perform_relnet_placement
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import perform_translation_layer_placement
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import perform_final_MLP_placement
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import perform_readout_placement
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import create_lsnn_output_spike_generators
from nxsdk_modules.lsnn.apps.relnet.data.loihi_final_placement_debug_for_FF_check_real_input import create_input_mask_spike_generator

from nxsdk_modules.lsnn.apps.relnet.data.loihi_placement_param_search import MAX_N_SENTENCES
from IPython.core import ultratb
sys.excepthook = ultratb.FormattedTB(mode='Verbose',
     color_scheme='Linux', call_pdb=1)

In [3]:
## Task parameters
# set the sample size for the run (-> need to be consistent with SNIP)
batch_size = 250
# choose task of bAbI dataset (-> needs to have # batch_size of length max_len)
task = 'qa17'
# choose length of sentences
max_len = 2

# other parameters
n_input_steps = 14
n_sim_steps = 37

time_steps_between_mask_spikes = 17
steps_per_word = 10

In [4]:
def process_input(task, max_len, batch_size):
    """Generates input spikes for the network."""

    data_file_path = "data/full_dataset.p"
    with open(data_file_path, 'rb') as fin:
        data_dict = pickle.load(fin)

    # parameters
    num_words = 180
    max_sentence_len = 11
    max_story_len = 20

    input_period_len = max_sentence_len*steps_per_word + n_sim_steps - n_input_steps + 10  # for non-overlapping LSNN and Relnets

    story_spike_list = []
    query_spike_list = []
    answers = []
    sentence_exists_mask_list = []

    # only stories of length max_len
    ids = np.where(data_dict['by_task'][task]['story_lengths'] == max_len)

    for i in range(0, 250):

        story = data_dict['by_task'][task]['stories'][ids][i]
        query = data_dict['by_task'][task]['queries'][ids][i]
        answer = data_dict['by_task'][task]['predicted_answer'][ids][i]

        sentence_exists_mask = np.any(story, axis=1)

        spikes = np.zeros((max_story_len, input_period_len, num_words))
        query_spikes = np.zeros((input_period_len, num_words))

        # story spikes
        for i in range(0, 20):
            # spikes
            for num, neuron in enumerate(story[i]):
                if neuron > 0:
                    for s in range((num * steps_per_word), (num*steps_per_word) + steps_per_word):
                        spikes[i, s, neuron-1] = 1

        # query spikes
        for num, neuron in enumerate(query):
            if neuron > 0:
                for s in range((num * steps_per_word), (num*steps_per_word) + steps_per_word):
                    query_spikes[s, neuron-1] = 1

        answers.append(answer)
        story_spike_list.append(spikes)
        query_spike_list.append(query_spikes)
        sentence_exists_mask_list.append(sentence_exists_mask)

    story_spikes = np.concatenate(story_spike_list, axis=1)
    query_spikes = np.concatenate(query_spike_list, axis=0)
    sentence_exists_mask = np.stack(sentence_exists_mask_list)

    return story_spikes, query_spikes, sentence_exists_mask, answers

In [5]:
# import data
data_file_path = "data/full_dataset.p"
with open(data_file_path, 'rb') as fin:
    data_dict = pickle.load(fin)
    #plot_data = data_dict['plot_data']
    cell_data = data_dict['cell_data']
    
# needed for weights/placement
stories_LSNN_cell_data = cell_data['stories_LSNN_cell_data']
queries_LSNN_cell_data = cell_data['queries_LSNN_cell_data']
relational_function_cell_data = cell_data['relational_function_cell_data']
quantization_cell_data = cell_data['quantization_cell_data']
final_MLP_cell_data = cell_data['final_MLP_cell_data']
readout_cell_data = cell_data['readout_cell_data']

# get input spikes
stories_LSNN_input_spikes, queries_LSNN_input_spikes, sentence_exists_mask, target_answers = process_input(task, max_len, batch_size)

batch_period = int(stories_LSNN_input_spikes.shape[1] / batch_size)
assert batch_period == stories_LSNN_input_spikes.shape[1] / batch_size

lsnn_input_time_steps = batch_period - 10 + n_input_steps - n_sim_steps  # 330

## just for mask!!
max_n_sentences = 20

assert sentence_exists_mask.shape == (batch_size, max_n_sentences)

# without pause between samples
input_mask_spikes = np.zeros((batch_size, max_n_sentences, batch_period))
input_spike_times = np.arange(lsnn_input_time_steps-n_input_steps+1, lsnn_input_time_steps-n_input_steps+1+n_sim_steps, time_steps_between_mask_spikes)
for t in input_spike_times:
    input_mask_spikes[:, :, t] = 1.0
input_mask_spikes *= (sentence_exists_mask[:, :, None] == 0.0).astype(np.float32)
input_mask_spikes = np.transpose(input_mask_spikes, [0, 2, 1])
input_mask_spikes = np.reshape(input_mask_spikes, (batch_size*batch_period, max_n_sentences))

assert input_mask_spikes.shape[0] == stories_LSNN_input_spikes.shape[1]


# Here we get the core connection arrays as according to the placement algorithms
((lsnn_sentence_core_connection_array, sentence_n_cores_total),
 (lsnn_question_core_connection_array, question_n_cores_total),
 ((input_mask_core, input_mask_relay_cores, input_mask_to_relay_connections), _),
 (relnet_init_core_connection_array, relnet_init_n_cores),
 (relnet_intermediate_core_connection_array, relnet_intermediate_n_cores),
 (translation_layer_core_connection, translation_layer_n_cores),
 (final_MLP_core_connection, final_MLP_n_cores),
 (readout_core_connection, readout_n_cores)) = \
    get_relational_network_placement_on_loihi(cell_data,
                                              use_optimal_intermediate_placement=True,
                                              use_cores_for_input=False)

# Here we connect to the cell and create the network net
net = nx.NxNet()

########### figure out network size from input #########################
max_sentences = int(max_len)
n_sentence1 = max_sentences
n_sentence2 = max_sentences


############ LSNN and relay layer ############
# Here we place the sentence LSNN's according to the mapping created above
(sentence_lsnn_id_to_neuron_group_map, sentence_lsnn_neuron_group_array,
 sentence_lsnn_id_to_relay_neuron_group_map, sentence_relay_neuron_group_array,
 sentence_lsnn_id_to_spike_gen_map, sentence_spike_generator_array) = \
    perform_LSNN_placement(lsnn_sentence_core_connection_array,
                           stories_LSNN_cell_data,
                           stories_LSNN_input_spikes,
                           5.0, net, max_sentences)

# Here we place the question LSNN's according to the mapping created above
(question_lsnn_id_to_neuron_group_map, question_lsnn_neuron_group_array,
 question_lsnn_id_to_relay_neuron_group_map, question_relay_neuron_group_array,
 question_lsnn_id_to_spike_gen_map, question_spike_generator_array) = \
    perform_LSNN_placement(lsnn_question_core_connection_array,
                           queries_LSNN_cell_data,
                           queries_LSNN_input_spikes,
                           5.0, net)

# TODO: figure out how to parameterise the weights etc. to really be able to scale the network config
input_mask_spike_gen, input_mask_id_to_spike_gen_map = create_input_mask_spike_generator(input_mask_core,
                                                                                         input_mask_relay_cores,
                                                                                         input_mask_to_relay_connections,
                                                                                         input_mask_spikes,
                                                                                         net, max_sentences)

# map for the relational layer
combined_lsnn_id_to_output_spike_gen_map = {**sentence_lsnn_id_to_relay_neuron_group_map,
                                            **question_lsnn_id_to_relay_neuron_group_map,
                                            **input_mask_id_to_spike_gen_map}

assert all(x.compileParams.logicalCoreId == id_ for id_, x in sentence_lsnn_id_to_relay_neuron_group_map.items())
assert all(x.compileParams.logicalCoreId == id_ for id_, x in question_lsnn_id_to_relay_neuron_group_map.items())


############ relational FF layers ############
relational_function_neuron_group_arrays = []
relational_function_id_to_neuron_group_maps = []

# layer 1 of the FF network
input_id_to_neuron_group_map = combined_lsnn_id_to_output_spike_gen_map

(relnet_neuron_group_array, relnet_init_id_to_neuron_group_map) = perform_relnet_placement(input_id_to_neuron_group_map,
                                                                                           relnet_init_core_connection_array,
                                                                                           relational_function_cell_data[0],
                                                                                           net, n_sentence1, n_sentence2)

relational_function_neuron_group_arrays.append(relnet_neuron_group_array)
relational_function_id_to_neuron_group_maps.append(relnet_init_id_to_neuron_group_map)

# layer 2 of the FF network
(relnet_neuron_group_array, relnet_id_to_neuron_group_map) =     perform_relnet_placement(relnet_init_id_to_neuron_group_map,
                             relnet_intermediate_core_connection_array[0],
                             relational_function_cell_data[1],
                             net, n_sentence1, n_sentence2)

relational_function_neuron_group_arrays.append(relnet_neuron_group_array)
relational_function_id_to_neuron_group_maps.append(relnet_id_to_neuron_group_map)

# layer 3 of the FF network
(relnet_neuron_group_array, relnet_id_to_neuron_group_map) =     perform_relnet_placement(relnet_id_to_neuron_group_map,
                             relnet_intermediate_core_connection_array[1],
                             relational_function_cell_data[2],
                             net, n_sentence1, n_sentence2)

relational_function_neuron_group_arrays.append(relnet_neuron_group_array)
relational_function_id_to_neuron_group_maps.append(relnet_id_to_neuron_group_map)

# layer 4 of the FF network
(relnet_neuron_group_array, relnet_id_to_neuron_group_map) =     perform_relnet_placement(relnet_id_to_neuron_group_map,
                             relnet_intermediate_core_connection_array[2],
                             relational_function_cell_data[3],
                             net, n_sentence1, n_sentence2)

relational_function_neuron_group_arrays.append(relnet_neuron_group_array)
relational_function_id_to_neuron_group_maps.append(relnet_id_to_neuron_group_map)

############ translational layer ############
(translation_layer_neuron_group_tuple,
 translation_layer_id_to_neuron_group_map) = \
    perform_translation_layer_placement(
        relational_function_id_to_neuron_group_maps[-1],
        translation_layer_core_connection,
        quantization_cell_data, net, n_sentence1, n_sentence2)

input_id_to_neuron_group_map = translation_layer_id_to_neuron_group_map

############ Readout FF layers ############
final_MLP_neuron_group_list = []
for i in range(len(final_MLP_cell_data)):
    (final_MLP_neuron_group_tuple,
     final_MLP_id_to_neuron_group_map) = perform_final_MLP_placement(input_id_to_neuron_group_map,
                                                                     final_MLP_core_connection[i],
                                                                     final_MLP_cell_data[i], net)
    final_MLP_neuron_group_list.append(final_MLP_neuron_group_tuple)
    input_id_to_neuron_group_map = final_MLP_id_to_neuron_group_map

############ final readout layer ############
(readout_neuron_group_tuple,
 readout_id_to_neuron_group_map) = \
    perform_readout_placement(input_id_to_neuron_group_map,
                              readout_core_connection,
                              readout_cell_data, net)

/home/pplank/nxsdk-nxsdk-submission/nxsdk-nxsdk/nxsdk_modules/lsnn/apps/relnet/data/loihi_chip_optimum_network_from_data.py:565: RuntimeWarning: divide by zero encountered in floor_divide
  fanout_count = output_axon_count//neuron_count


In [6]:
relay_sentences_core_ids = np.concatenate([np.concatenate([np.array([core.id for core in relay_copy])
                                                           for relay_copy in x.cores.relay])
                                           for x in lsnn_sentence_core_connection_array.ravel()])

In [7]:
# get core ids of lsnn and relay networks of sentences and questions for the SNIPs
lsnn_sentences_core_ids = [[core.id for core in x.cores.lsnn] for x in lsnn_sentence_core_connection_array.ravel()]
relay_sentences_core_ids = [[[core.id for core in relay_copy] for relay_copy in x.cores.relay] for x in lsnn_sentence_core_connection_array.ravel()]

lsnn_questions_core_ids = [core.id for core in lsnn_question_core_connection_array.cores.lsnn]
relay_questions_core_ids = [[core.id for core in relay_copy] for relay_copy in lsnn_question_core_connection_array.cores.relay]

lsnn_sentences_core_ids = np.array(lsnn_sentences_core_ids).ravel()
lsnn_questions_core_ids = np.array(lsnn_questions_core_ids).ravel()
relay_sentences_core_ids = np.array(relay_sentences_core_ids).ravel()
relay_questions_core_ids = np.array(relay_questions_core_ids).ravel()

## new
relay_sentences_core_ids = np.concatenate([np.concatenate([np.array([core.id for core in relay_copy])
                                                           for relay_copy in x.cores.relay])
                                           for x in lsnn_sentence_core_connection_array.ravel()])

relay_questions_core_ids = np.concatenate([np.array([core.id for core in relay_copy]) 
                                           for relay_copy in lsnn_question_core_connection_array.cores.relay])

lsnn_core_ids_list = [lsnn_sentences_core_ids, lsnn_questions_core_ids]
relay_core_ids_list = [relay_sentences_core_ids, relay_questions_core_ids]

# get readout core ids
readout_core_ids = [core.id for core in readout_core_connection.cores]
readout_core_ids_list = [np.array(readout_core_ids).ravel()]

<ipython-input-7-6401f275bf46>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  relay_sentences_core_ids = np.array(relay_sentences_core_ids).ravel()
<ipython-input-7-6401f275bf46>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  relay_questions_core_ids = np.array(relay_questions_core_ids).ravel()


In [8]:
# all core ids which are not LSNNs or relay neurons (for reset after classification)
assert len(readout_core_ids) == 1
highest_core_id = readout_core_ids[0]
lowest_core_id = 0 

lsnn_core_ids = np.concatenate(lsnn_core_ids_list)
relay_core_ids = np.concatenate(relay_core_ids_list)

lsnn_relay_core_ids = np.concatenate([lsnn_core_ids, relay_core_ids])

# these core ids needs to be reset
reset_core_ids = []
for i in range(lowest_core_id, highest_core_id):
    if i not in lsnn_relay_core_ids:
        reset_core_ids.append(i)

reset_core_ids_list = [np.array(reset_core_ids).ravel()]

In [9]:
# get chip_id and chip_core_id from core_ids
def get_chip_core_dict(core_ids_list):
    chip_core_dict = {}

    for ids in core_ids_list:
        chip_ids = np.floor(ids/128).astype(np.int32)
        chip_core_ids = ids%128
        
        for chip in range(0, np.max(chip_ids)+1):
            if chip in chip_core_dict.keys():
                chip_core_dict[chip].extend(chip_core_ids[np.where(chip_ids==chip)])
            else:
                if len(chip_core_ids[np.where(chip_ids==chip)].tolist()) > 0:
                    chip_core_dict[chip] = chip_core_ids[np.where(chip_ids==chip)].tolist()
                
    return chip_core_dict

lsnn_chip_core_dict = get_chip_core_dict(lsnn_core_ids_list)
relay_chip_core_dict = get_chip_core_dict(relay_core_ids_list)
readout_chip_core_dict = get_chip_core_dict(readout_core_ids_list)
reset_chip_core_dict = get_chip_core_dict(reset_core_ids_list)

In [10]:
def configure_lsnn_relay_snips(core_id_list, snip_dir):
    # Create SNIP for LSNN/relay procedure
    lsnn_channel_dict = {}
    relay_channel_dict = {}
        
    for chip_id in set(core_id_list):
        mgmtSnip = board.createSnip(
            name="runMgmt",
            includeDir=snip_dir,
            cFilePath=snip_dir + "snip_mgmt.c",
            funcName="run_mgmt", 
            guardName="do_mgmt", 
            phase=Phase.EMBEDDED_MGMT,
            chipId=chip_id)

        # Create a channel from super host to Lakemont for sending core ids of lsnn networks
        lsnn_channel = board.createChannel(bytes('lsnn_id_ch_' + str(chip_id), 'utf-8'), numElements=1,
                                           messageSize=128*4)
        lsnn_channel.connect(None, mgmtSnip)

        # Create a channel from super host to Lakemont for sending core ids of relay networks
        relay_channel = board.createChannel(bytes('relay_id_ch_' + str(chip_id), 'utf-8'), numElements=1,
                                            messageSize=128*4)
        relay_channel.connect(None, mgmtSnip)

        lsnn_channel_dict[chip_id] = lsnn_channel
        relay_channel_dict[chip_id] = relay_channel
    
    return lsnn_channel_dict, relay_channel_dict

In [11]:
def configure_readout_snips(reaout_core_id_list, readout_chip_core_dict, snip_dir):
    # Create SNIP for readout procedure
    
    readout_channel_dict = {}
    
    for chip_id in set(reaout_core_id_list):
        mgmtSnip = board.createSnip(
            name="runMgmt",
            includeDir=snip_dir,
            cFilePath=snip_dir + "readout_snip_mgmt.c",
            funcName="run_mgmt", 
            guardName="do_mgmt", 
            phase=Phase.EMBEDDED_MGMT,
            chipId=chip_id)

        # Create a channel from super host to Lakemont for sending core ids of lsnn networks
        if len(readout_chip_core_dict[chip_id]) > 0:
            readout_channel = board.createChannel(bytes('readout_id_ch_' + str(chip_id), 'utf-8'), numElements=1,
                                               messageSize=128*4)
            readout_channel.connect(None, mgmtSnip)
        else:
            readout_channel = None

        readout_channel_dict[chip_id] = readout_channel
        
        # configure init channel
        num_output = 181
        readout_init_channel = board.createChannel(b'output_neurons', numElements=4 * num_output, messageSize=4)
        readout_init_channel.connect(None, mgmtSnip)

        # configure classification channel
        classification_channel = board.createChannel(b'classifications', numElements=1, messageSize=batch_size*4)
        classification_channel.connect(mgmtSnip, None)
        
    return readout_channel_dict, readout_init_channel, classification_channel

In [12]:
def configure_reset_snips(reset_chip_core_dict, snip_dir):
    # Create SNIP for reset procedure
    
    reset_channel_dict = {}
    
    for chip_id in set(reset_chip_core_dict.keys()):
        mgmtSnip = board.createSnip(
            name="runMgmt",
            includeDir=snip_dir,
            cFilePath=snip_dir + "snip_reset.c",
            funcName="run_mgmt", 
            guardName="do_mgmt", 
            phase=Phase.EMBEDDED_MGMT,
            chipId=chip_id,
            lmtId=1)

        # Create a channel from super host to Lakemont for sending core ids which should be reset
        if len(reset_chip_core_dict[chip_id]) > 0:
            readout_channel = board.createChannel(bytes('reset_id_ch_' + str(chip_id), 'utf-8'), numElements=1,
                                               messageSize=128*4)
            readout_channel.connect(None, mgmtSnip)
        else:
            readout_channel = None

        reset_channel_dict[chip_id] = readout_channel
    
    return reset_channel_dict

In [13]:
def lsnn_relay_channel_write(core_id_list, lsnn_chip_core_dict, relay_chip_core_dict, lsnn_channel_dict, relay_channel_dict):
    default = list(np.zeros(128, dtype=np.int32) - 1)
    # write core ids to the channels
    for chip_id in set(core_id_list):
        # print(chip_id)
        if chip_id in lsnn_chip_core_dict:
            # print((lsnn_chip_core_dict[chip_id] + default)[:128])
            lsnn_channel_dict[chip_id].write(1, (lsnn_chip_core_dict[chip_id] + default)[:128])
        else:
            # there needs to be written something, otherwise the SNIP stalls
            lsnn_channel_dict[chip_id].write(1, default)
        if chip_id in relay_chip_core_dict:
            # print((relay_chip_core_dict[chip_id] + default)[:128])
            relay_channel_dict[chip_id].write(1, (relay_chip_core_dict[chip_id] + default)[:128])
        else:
            # there needs to be written something, otherwise the SNIP stalls
            relay_channel_dict[chip_id].write(1, default)

In [14]:
def readout_channel_write(readout_chip_core_dict, readout_channel_dict, readout_init_channel):
    default = list(np.zeros(128, dtype=np.int32) - 1)
    chip_id_list = list(readout_chip_core_dict.keys())
    for chip_id in set(chip_id_list):
        readout_channel_dict[chip_id].write(1, (readout_chip_core_dict[chip_id] + default)[:128])
        
        # TODO: change to a more flexible layout if output neurons do not fit on one core/chip
        board_id = 0
        core_id = readout_chip_core_dict[chip_id][0]  # Assumes only 1 core!
        num_output = 181
        for cx_id in range(num_output):
            readout_init_channel.write(4, [board_id, chip_id, core_id, cx_id])

In [15]:
def reset_channel_write(reset_chip_core_dict, reset_channel_dict):
    default = list(np.zeros(128, dtype=np.int32) - 1)
    chip_id_list = list(reset_chip_core_dict.keys())
    
    for chip_id in set(chip_id_list):
        reset_channel_dict[chip_id].write(1, (reset_chip_core_dict[chip_id] + default)[:128])

In [16]:
# remove chip 0 and chip 21 from the reset list (reset handled in their respective SNIP)
del reset_chip_core_dict[0]
del reset_chip_core_dict[21]

In [17]:
runtime = batch_size * batch_period + 2

In [18]:
with setEnvWithinContext(PARTITION='nahuku32', BOARD='ncl-ghrd-01'):
    # compile network
    board = nx.N2Compiler().compile(net)

    # Define directory where SNIP C-code is located
    snip_dir = os.getcwd() + "/snips/"
    
    # Configure LSNN/relay SNIP
    core_id_list = list(lsnn_chip_core_dict.keys()) + list(relay_chip_core_dict.keys())
    lsnn_channel_dict, relay_channel_dict = configure_lsnn_relay_snips(core_id_list, snip_dir)
    
    # Configure readout SNIP
    readout_channel_dict, readout_init_channel, classification_channel = configure_readout_snips(readout_chip_core_dict, 
                                                                                                 readout_chip_core_dict, 
                                                                                                 snip_dir)
    
    # Configure reset SNIPs (all other)
    reset_channel_dict = configure_reset_snips(reset_chip_core_dict, snip_dir)
        
    # Performance probes
    buffer_size = 1024
    bin_size = int(2**np.ceil(np.log2(runtime/buffer_size))) # should be a power of 2

    etProbe = board.probe(
    probeType=ProbeParameter.ENERGY, 
    probeCondition=PerformanceProbeCondition(
        tStart=1, 
        tEnd=runtime, 
        bufferSize=buffer_size, 
        binSize=bin_size)
    )
     
    # Start driver
    board.start()
    
    # write into channels for LSNN/relay SNIP
    lsnn_relay_channel_write(core_id_list, lsnn_chip_core_dict, relay_chip_core_dict, lsnn_channel_dict, relay_channel_dict)
    
    # write into channels for readout SNIP
    readout_channel_write(readout_chip_core_dict, readout_channel_dict, readout_init_channel)
    
    # write into channels for reset SNIP
    reset_channel_write(reset_chip_core_dict, reset_channel_dict)
        
    # run network
    board.run(runtime, True)

    # read classifications
    classifications = []
    classifications.append(classification_channel.read(1))
    
    board.finishRun()
    board.disconnect()

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.22:45329
INFO:DRV:      Host server up..............Done 0.25s
INFO:DRV:      Encoding axons/synapses.....Done 0.87s
INFO:DRV:      Compiling Embedded snips....Done 16.50s
INFO:DRV:      Compiling MPDS Registers....Done 1.52ms
INFO:DRV:      Compiling Register Probes...Done 0.76ms
INFO:DRV:      Compiling Spike Probes......Done 0.63ms
INFO:HST:  Args chip=0 cpu=0 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip0_lmt0.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=1 cpu=0 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip1_lmt0.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=1 cpu=1 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip

INFO:HST:  Args chip=16 cpu=1 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip16_lmt1.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=17 cpu=0 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip17_lmt0.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=17 cpu=1 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip17_lmt1.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=18 cpu=0 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1628351903.884258/launcher_chip18_lmt0.bin --chips=22 --remote-relay=0 --epoch=0 --cpu-active-ratio=1 
INFO:HST:  Args chip=18 cpu=1 /home/pplank/lava_env/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/162

In [19]:
# get predictions
predictions = classifications[0]

In [20]:
result = len(set(target_answers) & set(predictions)) / float(len(set(target_answers) | set(predictions))) * 100
print("Classification accuracy of task {} for {} samples of lenght {}: {}%".format(task, batch_size, max_len, result))

Classification accuracy of task qa17 for 250 samples of lenght 2: 100.0%


In [21]:
total = etProbe.totalTimePerTimeStep[0:runtime]
host = etProbe.hostTimePerTimeStep[0:runtime]

effective = total - host

latency = np.sum(effective) / batch_size
print('Total number of time steps: ', runtime)
print('Total time (µs):', np.sum(total))
print('Mean time per time step (µs):', np.mean(total))
print('Time per inference (µs) ({} samples): {}'.format(batch_size, latency))

# EDP
stats = board.energyTimeMonitor.powerProfileStats

n_dynamic = stats['power']['core']['dynamic']
n_static = stats['power']['core']['static']

x86_dynamic = stats['power']['lakemont']['dynamic']
x86_static = stats['power']['lakemont']['static']

dynamic = n_dynamic + x86_dynamic
static = n_static + x86_static

total = dynamic + static
energy = latency * total / 10**6
edp = energy * latency / 1000
print()
print("Dynamic power (mW): ", dynamic)
print("Static power (mW): ", static)
print("Total power (mW): ", total)
print()
print("Latency per inference (ms): ", latency / 1000)
print("Total energy per inference (mJ): ", energy)
print("EDP (µJs): ", edp)
print()
print("Power in detail:")
print("x86 static power (mW): ", x86_static)
print("x86 dynamic power (mW): ", x86_dynamic)
print("Core static power (mW): ", n_static)
print("Core dynamic power (mW): ", n_dynamic)

Total number of time steps:  35752
Total time (µs): 2276299.999999985
Mean time per time step (µs): 63.66916536137797
Time per inference (µs) (250 samples): 9105.19999999994

Dynamic power (mW):  630.0983325449383
Static power (mW):  191.14990814809457
Total power (mW):  821.2482406930329

Latency per inference (ms):  9.105199999999941
Total energy per inference (mJ):  7.477629481158155
EDP (µJs):  68.0853119518408

Power in detail:
x86 static power (mW):  9.541154247711152
x86 dynamic power (mW):  407.4348348767732
Core static power (mW):  181.60875390038342
Core dynamic power (mW):  222.66349766816512
